In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving beats_reviews.csv to beats_reviews (2).csv


In [ ]:
df = pd.read_csv('beats_reviews.csv')

In [ ]:
print(df.head())

         asin       review_id  \
0  B0D4SX9RC6  R25X0TE59JOMS3   
1  B0D4SX9RC6   RA0QY335MJ1GL   
2  B0D4SX9RC6  R3GFKAF488MHHP   
3  B0D4SX9RC6  R2SW6Y89GDH3FG   
4  B0D4SX9RC6  R3R7XPH8WUAWNW   

                                               title         author  rating  \
0  5.0 out of 5 stars Used Beats Pill 2024 still ...  Leila L’Abate       5   
1  5.0 out of 5 stars So much power in a compact ...         LyneeZ       5   
2                 5.0 out of 5 stars Worth the money  Lauren Bollen       5   
3  5.0 out of 5 stars I'm finding I have to charg...    wanda lyons       5   
4       5.0 out of 5 stars The Kim Gray is excellent         Butter       5   

                                             content  \
0  I bought this used in "acceptable" condition, ...   
1  Just like most others who have reviewed this i...   
2  I’m not going to lie, I wasn’t a skeptic at fi...   
3  Hello. This product is awesome. I listen to mu...   
4  This is a great sounding Bluetooth speaker.

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   asin                42 non-null     object
 1   review_id           42 non-null     object
 2   title               42 non-null     object
 3   author              42 non-null     object
 4   rating              42 non-null     int64 
 5   content             42 non-null     object
 6   timestamp           42 non-null     object
 7   profile_id          27 non-null     object
 8   is_verified         42 non-null     bool  
 9   helpful_count       42 non-null     int64 
 10  product_attributes  41 non-null     object
dtypes: bool(1), int64(2), object(8)
memory usage: 3.5+ KB
None


In [ ]:
print(df.isnull())

     asin  review_id  title  author  rating  content  timestamp  profile_id  \
0   False      False  False   False   False    False      False       False   
1   False      False  False   False   False    False      False       False   
2   False      False  False   False   False    False      False       False   
3   False      False  False   False   False    False      False       False   
4   False      False  False   False   False    False      False       False   
5   False      False  False   False   False    False      False       False   
6   False      False  False   False   False    False      False       False   
7   False      False  False   False   False    False      False       False   
8   False      False  False   False   False    False      False        True   
9   False      False  False   False   False    False      False        True   
10  False      False  False   False   False    False      False        True   
11  False      False  False   False   False    False

In [ ]:
#Check for duplicates
print(df.duplicated())

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
dtype: bool


In [ ]:
#Date
df['clean_date'] = df['timestamp'].str.extract(r'([A-Za-z]+ \d{1,2}, \d{4})')
df['review_date'] = pd.to_datetime(df['clean_date'], format='%B %d, %Y', errors='coerce')
df.drop(columns=['clean_date'], inplace=True)

In [ ]:
#Split product attributes
df['Color'] = df['product_attributes'].str.extract(r'Color: ([^,]+)')
df['Style'] = df['product_attributes'].str.extract(r'Style: ([^,]+)')
df['Set'] = df['product_attributes'].str.extract(r'Set: (.+)')

In [ ]:
columns_order = ['asin', 'review_id', 'review_date', 'rating', 'title', 'author',
                 'content', 'Color', 'Style', 'Set', 'is_verified', 'helpful_count',
                 'profile_id', 'product_attributes', 'timestamp']
df = df[[col for col in columns_order if col in df.columns]]

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   asin                42 non-null     object        
 1   review_id           42 non-null     object        
 2   review_date         42 non-null     datetime64[ns]
 3   rating              42 non-null     int64         
 4   title               42 non-null     object        
 5   author              42 non-null     object        
 6   content             42 non-null     object        
 7   Color               41 non-null     object        
 8   Style               21 non-null     object        
 9   Set                 13 non-null     object        
 10  is_verified         42 non-null     bool          
 11  helpful_count       42 non-null     int64         
 12  profile_id          27 non-null     object        
 13  product_attributes  41 non-null     object        
 

In [ ]:
df.to_csv("beats_reviews_cleaned.csv", index=False)